# Web Scrapping

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import sqlite3

#### Hacemos web scrapping de una página para obtener la lista con la que haremos el scrapping principal

In [3]:
url_list = 'https://pokemondb.net/pokedex/all'
list_response = requests.get(url_list)
if list_response:
    list_soup = BeautifulSoup(list_response.text, 'html')

pokenombre = list_soup.find_all('a', class_= 'ent-name')
pokelista = [name.get_text().strip() for name in pokenombre]
pokeset = set(pokelista)
pokeset

{'Araquanid',
 'Frosmoth',
 'Regigigas',
 'Braixen',
 'Baltoy',
 'Jellicent',
 'Celesteela',
 'Throh',
 'Relicanth',
 'Diancie',
 'Happiny',
 'Marowak',
 'Toucannon',
 'Gumshoos',
 'Snorunt',
 'Vikavolt',
 'Magcargo',
 'Whiscash',
 'Feraligatr',
 'Pachirisu',
 'Dracozolt',
 'Gabite',
 'Florges',
 'Sharpedo',
 'Lumineon',
 'Lileep',
 'Bibarel',
 'Purugly',
 'Haunter',
 'Bellibolt',
 'Glimmet',
 'Vanillite',
 'Iron Leaves',
 'Scorbunny',
 'Ivysaur',
 'Pansear',
 'Feebas',
 'Stantler',
 'Riolu',
 'Victreebel',
 'Solosis',
 'Slaking',
 'Guzzlord',
 'Nosepass',
 'Sudowoodo',
 'Boldore',
 'Fletchling',
 'Yveltal',
 'Arctozolt',
 'Mabosstiff',
 'Passimian',
 'Zamazenta',
 'Omastar',
 'Musharna',
 'Venusaur',
 'Scraggy',
 'Jolteon',
 'Koffing',
 'Empoleon',
 'Carnivine',
 'Shaymin',
 'Wooloo',
 'Larvitar',
 'Kangaskhan',
 'Orbeetle',
 'Seadra',
 'Dodrio',
 'Elekid',
 'Duskull',
 'Bellsprout',
 'Thievul',
 'Taillow',
 'Huntail',
 'Sinistea',
 'Flutter Mane',
 'Chinchou',
 'Wishiwashi',
 'Fearow

### Inicializamos una lista de la que sacaremos el DataFrame

In [4]:
ds = []

### Hacemos Web Scrapping saltando de página en página con los elementos obtenidos antes y añadimos a la lista como diccionarios

In [5]:
for n in pokeset:
    resourece_url = f'https://pokemondb.net/pokedex/{n}'

    response = requests.get(resourece_url)
    
    if response:
        soup = BeautifulSoup(response.text, 'html')

    number_element = soup.find("th", string="National №").find_next_sibling("td")
    number = int(number_element.text.strip())
    name = n
    height_element = soup.find(string="Height").find_next()
    height = re.search(r"\d+\.\d+", height_element.text).group()
    weight_element = soup.find(string="Weight").find_next()
    weight = re.search(r"\d+\.\d+", weight_element.text).group()
    type_elements = soup.find("th", string="Type").find_next_sibling("td").find_all("a")
    types = [t.text for t in type_elements]
    types_str = ", ".join(types)  # Para mostrarlo como cadena
    abilities_element = soup.find("th", string="Abilities").find_next_sibling("td").find("a")
    first_ability = abilities_element.text.strip()
    ds.append({'Name': n, 
               'Number' : number, 
               'Height (m)' : height, 
               'Weight (Kg)': weight, 
               'Types' : types_str, 
               'Ability' : first_ability})


In [6]:
ds

[{'Name': 'Araquanid',
  'Number': 752,
  'Height (m)': '1.8',
  'Weight (Kg)': '82.0',
  'Types': 'Water, Bug',
  'Ability': 'Water Bubble'},
 {'Name': 'Frosmoth',
  'Number': 873,
  'Height (m)': '1.3',
  'Weight (Kg)': '42.0',
  'Types': 'Ice, Bug',
  'Ability': 'Shield Dust'},
 {'Name': 'Regigigas',
  'Number': 486,
  'Height (m)': '3.7',
  'Weight (Kg)': '420.0',
  'Types': 'Normal',
  'Ability': 'Slow Start'},
 {'Name': 'Braixen',
  'Number': 654,
  'Height (m)': '1.0',
  'Weight (Kg)': '14.5',
  'Types': 'Fire',
  'Ability': 'Blaze'},
 {'Name': 'Baltoy',
  'Number': 343,
  'Height (m)': '0.5',
  'Weight (Kg)': '21.5',
  'Types': 'Ground, Psychic',
  'Ability': 'Levitate'},
 {'Name': 'Jellicent',
  'Number': 593,
  'Height (m)': '2.2',
  'Weight (Kg)': '135.0',
  'Types': 'Water, Ghost',
  'Ability': 'Water Absorb'},
 {'Name': 'Celesteela',
  'Number': 797,
  'Height (m)': '9.2',
  'Weight (Kg)': '999.9',
  'Types': 'Steel, Flying',
  'Ability': 'Beast Boost'},
 {'Name': 'Throh',

### Convertimos la lista de diccionarios en DataFrame y lo ordenamos por Número de Pokédex, poniendolo además como indice

In [7]:

df = pd.DataFrame(ds)
df = df.sort_values(by = ['Number'])
df.set_index('Number', drop = False, inplace = True)
df

,Name,Number,Height (m),Weight (Kg),Types,Ability
Number,,,,,,
1,Bulbasaur,1,0.7,6.9,"Grass, Poison",Overgrow
1,Sandy Shocks,1,0.7,6.9,"Grass, Poison",Overgrow
2,Ivysaur,2,1.0,13.0,"Grass, Poison",Overgrow
3,Venusaur,3,2.0,100.0,"Grass, Poison",Overgrow
4,Charmander,4,0.6,8.5,Fire,Blaze
...,...,...,...,...,...,...
1018,Archaludon,1018,2.0,60.0,"Steel, Dragon",Stamina
1019,Hydrapple,1019,1.8,93.0,"Grass, Dragon",Supersweet Syrup
1024,Terapagos,1024,0.2,6.5,Normal,Tera Shift


### Conectamos con SQLite y creamos la tabla con la información obtenida

In [8]:
conn = sqlite3.connect('pokedex.db')
df.to_sql('pokedex', conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

## Conclusion 

Se ha implementado una base de datos en formato SQL mediante el motor SQLite, la cual contiene información detallada de los 1025 Pokémon actualmente registrados en la Pokédex. Para la construcción de esta base de datos, se utilizó un DataFrame de Pandas generado a partir de una lista de diccionarios. Los datos fueron obtenidos mediante técnicas de web scraping aplicadas a la página web https://pokemondb.net/pokedex/. En primer lugar, se extrajeron los nombres de los Pokémon desde https://pokemondb.net/pokedex/all, donde se encuentran listados los 1025 especímenes. A continuación, se accedió programáticamente a las páginas individuales de cada Pokémon a través de la URL https://pokemondb.net/pokedex/{nombre}, con el fin de recopilar atributos como el número de Pokédex, altura, peso, tipos y habilidad principal (en idioma inglés). Dado que los nombres iniciales no se encontraban en orden, el DataFrame fue ordenado posteriormente en función del número de Pokédex, el cual también fue establecido como índice del mismo.